In [1]:
'''
survey_kernel_tutorial.ipynb

Notebook containing WL kernel similarity tutorial for survey paper
that operates on example RAx group of chemicals. Contains code from Oxford Protein Informatics Group to featureize
RDKit graphs.

QA ID: I-CCED-0032994

Author: Brett Hagan

PI: Grace Patlewicz

Last modified: 3/14/24

'''

'\nsurvey_kernel_tutorial.ipynb\n\nNotebook containing WL kernel similarity tutorial for survey paper\nthat operates on example RAx group of chemicals. Contains code from Oxford Protein Informatics Group to featureize\nRDKit graphs.\n\nQA ID: I-CCED-0032994\n\nAuthor: Brett Hagan\n\nPI: Grace Patlewicz\n\nLast modified: 3/14/24\n\n'

In [1]:
import matplotlib.pyplot as plt
import grakel
from rdkit import Chem
import numpy as np
import pandas as pd
import networkx as nx

In [16]:
# list of dtxsids and smiles from category
dtxsids = ['2-Amino-4,6-Dinitrotoluene',
'2,4,6-Trinitrotoluene',
'2-Methyl-5-nitroaniline',
'Isopropalin',
'Pendimethalin',
'Trifluralin'
]

smiles = ['CC1=C(C=C(C=C1N)[N+]([O-])=O)[N+]([O-])=O',
'CC1=C(C=C(C=C1[N+]([O-])=O)[N+]([O-])=O)[N+]([O-])=O',
'CC1=C(N)C=C(C=C1)[N+]([O-])=O',
'CCCN(CCC)C1=C(C=C(C=C1[N+]([O-])=O)C(C)C)[N+]([O-])=O',
'CCC(CC)NC1=C(C=C(C)C(C)=C1[N+]([O-])=O)[N+]([O-])=O',
'CCCN(CCC)C1=C(C=C(C=C1[N+]([O-])=O)C(F)(F)F)[N+]([O-])=O'
]

In [21]:
# function to create molecular graphs from SMILES

def smile_to_mol_graph(smile):
    mol = Chem.MolFromSmiles(smile)
    g = nx.Graph()

    # Add nodes with atom symbols
    for atom in mol.GetAtoms():
        g.add_node(atom.GetIdx(), atom_symbol=atom.GetSymbol())
    
    # Add edges (bonds) with bond types
    for bond in mol.GetBonds():
        g.add_edge(bond.GetBeginAtomIdx(),
                   bond.GetEndAtomIdx(),
                   bond_type=bond.GetBondType())
    return g

In [26]:
# conver to grakel graph objects with atom symbol as feature

graphs = [smile_to_mol_graph(smile) for smile in smiles]
grakel_graphs = grakel.graph_from_networkx(graphs,node_labels_tag='atom_symbol')

In [23]:
graphs

In [24]:
# define WL kernel
# highest similarity corresponds to correct RAx source analogue
# visualize in notebook

wl_kernel = grakel.WeisfeilerLehman(base_graph_kernel=grakel.VertexHistogram,normalize=True)
p = wl_kernel.fit_transform(grakel_graphs)
df = pd.DataFrame(p)
display(df)

,0,1,2,3,4,5
0,1.000000,0.877781,0.734344,0.656103,0.703633,0.616602
1,0.877781,1.000000,0.633474,0.597295,0.646695,0.569431
2,0.734344,0.633474,1.000000,0.670563,0.699910,0.616493
3,0.656103,0.597295,0.670563,1.000000,0.763393,0.853267
4,0.703633,0.646695,0.699910,0.763393,1.000000,0.696215
5,0.616602,0.569431,0.616493,0.853267,0.696215,1.000000


In [27]:
from grakel import GraphKernel
# Initialize Graphlet Kernel
gk = GraphKernel(kernel=["graphlet_sampling"], normalize=True)

# Compute Kernel Matrix
kernel_matrix = gk.fit_transform(grakel_graphs)


In [29]:
pd.DataFrame(kernel_matrix)

,0,1,2,3,4,5
0,1.000000,0.993798,0.993249,0.977407,0.987774,0.987481
1,0.993798,1.000000,0.999988,0.994849,0.998983,0.998793
2,0.993249,0.999988,1.000000,0.995327,0.999189,0.999004
3,0.977407,0.994849,0.995327,1.000000,0.998408,0.998313
4,0.987774,0.998983,0.999189,0.998408,1.000000,0.999852
5,0.987481,0.998793,0.999004,0.998313,0.999852,1.000000


Graphlet Kernel: Focuses on counting small connected subgraphs (graphlets) within the molecular graphs. This could add fine-grained comparison for small datasets.

In [30]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 45.0 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 44.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 49.7 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
